In [1]:
import requests
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime

In [2]:
%matplotlib inline

In [3]:
TOKEN = r'd46da9b4388836c29c2728dcc7cd1dac63de131d'
def gh_request(url, *args, **kwargs):
    BASE = r'https://api.github.com'
    r = requests.get(BASE + url, *args, **kwargs)
    if r.status_code != 200 and not int(r.headers['X-RateLimit-Remaining']):
        reset = datetime.fromtimestamp(int(r.headers['X-RateLimit-Reset']))
        raise requests.HTTPError('Rate limit exceeded! Blocked until {}'.format(reset))
    r.raise_for_status()
    return r

In [4]:
def repos(owner):
    return gh_request('/users/{}/repos'.format(owner)).json()

def views_and_clones(owner, repo, **kw):
    views = gh_request('/repos/{}/{}/traffic/views'.format(owner, repo), **kw).json()
    clones = gh_request('/repos/{}/{}/traffic/clones'.format(owner, repo), **kw).json()
    releases = gh_request('/repos/{}/{}/releases'.format(owner, repo)).json()
    downloads = 0
    for release in releases:
        for asset in release['assets']:
            downloads += asset['download_count']
    return views['count'], clones['count'], downloads

def anaconda_downloads(owner, package, **kw):
    r = requests.get('https://api.anaconda.org/package/{}/{}'.format(owner, package))
    if r.status_code != 200:
        return 0
    downloads = 0
    for f in r.json()['files']:
        downloads += f['ndownloads']
    return downloads

def stats(owner, **kw):
    data = {}
    for repo in tqdm(repos(owner)):
        repo_owner = repo['owner']['login']
        repo_name = repo['name']
        stars = repo['stargazers_count']
        forks = repo['forks_count']
        anaconda_dl = anaconda_downloads(repo_owner, repo_name)
        views, clones, gh_downloads = views_and_clones(repo_owner, repo_name, **kw)
        data[repo_name] = {
            'GitHub Stars': stars,
            'GitHub Clones': clones,
            'GitHub Forks': forks,
            'GitHub Visits': views,
            'GitHub Downloads': gh_downloads,
            'Conda Installations': anaconda_dl,
            'PyPI Installations': 0,
            'WebApp Users': 0,
        }
    return data

In [5]:
insilichem_stats = stats('insilichem',  auth=('jaimergp', TOKEN))
jaimergp_stats = stats('jaimergp', auth=('jaimergp', TOKEN))

HTTPError: Rate limit exceeded! Blocked until 2018-07-16 18:49:25

# PyPI installations

Those must be obtained through (paid) Google Query, with the help of pypinfo

```
    > pypinfo --days 1000 --pip pychimera                          (base) 1.1m
    Served from cache: False
    Data processed: 255.49 GiB
    Data billed: 255.50 GiB
    Estimated cost: $1.25

    | download_count |
    | -------------- |
    |            435 |

    > pypinfo --days 1000 --pip ommprotocol                       (base) 39.5s
    Served from cache: False
    Data processed: 255.49 GiB
    Data billed: 255.50 GiB
    Estimated cost: $1.25

    | download_count |
    | -------------- |
    |             76 |

    > pypinfo --days 1000 --pip esigen                            (base) 40.4s
    Served from cache: False
    Data processed: 255.49 GiB
    Data billed: 255.50 GiB
    Estimated cost: $1.25

    | download_count |
    | -------------- |
    |             15 |

```

In [ ]:
insilichem_stats['easymecp'] = jaimergp_stats['easymecp']
insilichem_stats['pychimera']['PyPI Installations'] = 435
insilichem_stats['ommprotocol']['PyPI Installations'] = 76
insilichem_stats['esigen']['PyPI Installations'] = 15
insilichem_stats['esigen']['WebApp Users'] = 88  # From Google Analytics unique views to /configure URL

In [ ]:
df_raw_all = pd.DataFrame.from_dict(insilichem_stats, orient='index')
df_raw_all

In [ ]:
dfT = df_raw.T
for col in dfT:
    if col.startswith('tangram_'):
        dfT['tangram'] += dfT[col]

In [ ]:
for col in dfT:
    if col.startswith('tangram_'):
        dfT.drop(col, axis=1)


In [ ]:
dfTT = dfT.T

In [ ]:
df_raw = dfTT

In [ ]:
df = pd.DataFrame()
df['Code Impact'] = sum(df_raw[k] for k in ('GitHub Stars', 'GitHub Forks', 'GitHub Visits'))
df['Installations'] = sum(df_raw[k] for k in ('Conda Installations', 'PyPI Installations', 'WebApp Users', 'GitHub Clones', 'GitHub Downloads'))
df['sum'] = df.T.sum()
df.sort_values('sum', ascending=False, inplace=True)
df

In [ ]:
df2 = df[[((i not in ('libtangram', 'gaudiview', 'scripts', 'gaudinspect', 'insilichem.github.io', 'conda-recipes')) 
            and not i.startswith('tangram_')) for i in df2.index.values]]

In [ ]:
df2

In [ ]:
p = df2.drop('sum', axis=1).plot.bar(figsize=(30,15), fontsize=25, stacked=True)
p.legend(fontsize=25)
p

In [ ]:
p.figure.savefig('ghstats.pdf', bbox_inches='tight')